In [140]:
print('hello, world!')

hello, world!


In [141]:
import os

import datasets
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm

In [142]:
k_model_dir = "model/"
k_test_file = "data/test.source"
k_test_path = "test/"

k_base_model = "mrm8488/t5-base-finetuned-wikiSQL" 
k_base_model_name = "t5-base-finetuned-wikiSQL"
k_base_tokenizer = "mrm8488/t5-base-finetuned-wikiSQL" 
k_tune_tokenizer = "tokenizer/spider_original-01"

with open(k_test_file, "r") as file: 
    k_test = file.readlines()

k_model_dirs = [f for f in os.listdir(k_model_dir) if os.path.isdir(os.path.join(k_model_dir, f))]
k_model_paths = [os.path.join(k_model_dir, model_dir) for model_dir in k_model_dirs]

In [161]:
def process_in_batches(model, tokenizer, test_data, batch_size):
    predictions = []
    total_batches = len(test_data) // batch_size + (1 if len(test_data) % batch_size else 0)

    for i in tqdm(range(total_batches), desc="Processing", unit="batch"):
        batch = test_data[i * batch_size:(i + 1) * batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        output_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, max_length=512, no_repeat_ngram_size=2)
        batch_predictions = [tokenizer.decode(g, skip_special_tokens=True) for g in output_ids]
        predictions.extend(batch_predictions)
    return predictions

def test_model(test_data, model_path, tokenizer_path, test_path, model_name, batch_size, mock=False): 

    if mock: 
        write_path = f"{test_path}{model_name}.txt"
        print(f"model: {model_path}, tokenizer: {tokenizer_path}, test_path: {test_path}, model_name: {model_name}, write_path: {write_path}")
    else: 
        print(f"loading model from path: {model_path}")
        model = T5ForConditionalGeneration.from_pretrained(model_path)
        tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
        predictions = process_in_batches(model, tokenizer, test_data, batch_size)
    
        with open(f"{test_path}{model_name}.txt", "w") as file: 
            for pred in predictions:
                file.write(pred + '\n')

In [164]:
mock = False
test_model(
    test_data=k_test, 
    model_path=k_base_model, 
    tokenizer_path=k_base_tokenizer, 
    test_path=k_test_path, 
    model_name=k_base_model_name, 
    batch_size=8, 
    mock=mock
)

for i in range(len(k_model_dirs)): 
    model_path = k_model_paths[i]
    model_name = k_model_dirs[i]
    
    test_model(
        test_data=k_test, 
        model_path=model_path, 
        tokenizer_path=k_tune_tokenizer, 
        test_path=k_test_path, 
        model_name=model_name, 
        batch_size=8, 
        mock=mock
    )

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


loading model from path: mrm8488/t5-base-finetuned-wikiSQL


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Processing: 100%|██████████| 109/109 [09:27<00:00,  5.20s/batch]


loading model from path: model/spider_original-01


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing: 100%|██████████| 109/109 [05:17<00:00,  2.92s/batch]


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing: 100%|██████████| 109/109 [05:10<00:00,  2.85s/batch]


loading model from path: model/synthetic_joint-47


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing: 100%|██████████| 109/109 [05:29<00:00,  3.02s/batch]


loading model from path: model/synthetic_joint-49


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing: 100%|██████████| 109/109 [05:26<00:00,  3.00s/batch]


In [160]:
from transformers import T5ForConditionalGeneration
import torch


# Load the models from the saved checkpoint directories
model1 = T5ForConditionalGeneration.from_pretrained('model/synthetic_joint-01')
model2 = T5ForConditionalGeneration.from_pretrained('model/synthetic_joint-49')

# Function to compare two models
def are_models_identical(model1, model2):
    model1_dict = model1.state_dict()
    model2_dict = model2.state_dict()
    print(len(model1_dict))
    print(len(model2_dict))

    if set(model1_dict.keys()) != set(model2_dict.keys()):
        # Different sets of parameters
        return False

    count = 0 
    for param in model1_dict:
        if not torch.equal(model1_dict[param], model2_dict[param]):
            # Found a mismatch
            count += 1
            print('mismatch')
            return False

    return True

# Check if the models are the same
are_same = are_models_identical(model1, model2)

print(f"The models are {'the same' if are_same else 'different'}.")


260
260
mismatch
The models are different.
